In [1]:
from feature_extraction import fwrf_features
from utils import texture_utils, default_paths, nsd_utils
from model_fitting import initialize_fitting

import os, sys
import numpy as np
from PIL import Image

def get_top_patches_sketch_tokens(top_n_images=96,which_prf_grid=5,debug=False):

    """
    For each feature in sketch tokens feature space, identify the top n images for each pRF, 
    crop around each pRF and save the patches. 
    """

    subjects = np.arange(1,9)
    path_to_load = default_paths.sketch_token_feat_path
    feat_loaders = [fwrf_features.fwrf_feature_loader(subject=ss,\
                                                    which_prf_grid=which_prf_grid, \
                                                    feature_type='sketch_tokens',\
                                                    use_pca_feats = False) for ss in subjects]
    n_features = feat_loaders[0].max_features
    
    # putting all the patch images into this folder, can delete once we've analyzed them
    folder2save = os.path.join(default_paths.sketch_token_feat_path, 'top_im_patches')
    if not os.path.exists(folder2save):
        os.mkdir(folder2save)

    # how many images to look at per subj? will later combine across all subs.
    top_n_each_subj = int(np.ceil(top_n_images/len(subjects)))
    
    prf_models = initialize_fitting.get_prf_models(which_prf_grid)
    n_prfs = prf_models.shape[0]
    
    for ss, feat_loader in zip(subjects, feat_loaders):

        val_inds = np.array(nsd_utils.get_subj_df(subject=ss)['shared1000'])
        trninds = np.where(val_inds==False)[0]
        
        images = nsd_utils.get_image_data(subject=ss)
        images = images[trninds,:,:,:]
        image_size = images.shape[2:4]
        images = nsd_utils.image_uncolorize_fn(images)

        for mm in range(n_prfs):

            print('proc subj %d, prf %d of %d'%(ss, mm, n_prfs))
            if debug and mm>1:
                continue

            bbox = texture_utils.get_bbox_from_prf(prf_models[mm,:], image_size, n_prf_sd_out=2, \
                                     min_pix=None, verbose=False, force_square=False)

            feat, _ = feat_loader.load(trninds, prf_model_index=mm)
            assert(feat.shape[0]==images.shape[0])
            
            for ff in range(n_features):

                # sort in descending order, to get top n images
                sorted_order = np.flip(np.argsort(feat[:,ff]))
            
                top_images = images[sorted_order[0:top_n_each_subj],:,:,:]

                # taking just the patch around this pRF, because this is the region that 
                # contributed to computing the sketch tokens feature
                top_images_cropped = top_images[:,:,bbox[0]:bbox[1], bbox[2]:bbox[3]]

                for ii in range(top_n_each_subj):

                    image2save = (top_images_cropped[ii,0,:,:]*255).astype(np.uint8)
                    fn2save = os.path.join(folder2save, 'S%d_prf%d_feature%d_ranked%d.jpg'%(ss, mm, ff, ii))
                    print('saving to %s'%fn2save)
                    Image.fromarray(image2save).save(fn2save)

In [ ]:
get_top_patches_sketch_tokens(top_n_images=96,which_prf_grid=5,debug=True)


Loading images for subject 1

image data size: (10000, 3, 240, 240) , dtype: uint8 , value range: 0 255
proc subj 1, prf 0 of 1456
Loading pre-computed features for models [0 - 99] from /user_data/mmhender/features/sketch_tokens/S1_features_each_prf_grid5.h5py
Took 16.97199 seconds to load file
Size of features array for this image set is:
(9000, 150, 100)
Index into batch for prf 0: 0
Size of features array for this image set and prf is:
(9000, 150)
Final size of feature matrix is:
(9000, 150)
(21, 21)
[247, 45]
saving to /user_data/mmhender/features/sketch_tokens/top_im_patches/S1_prf0_feature0_ranked0.jpg
(21, 21)
[240, 38]
saving to /user_data/mmhender/features/sketch_tokens/top_im_patches/S1_prf0_feature0_ranked1.jpg
(21, 21)
[157, 3]
saving to /user_data/mmhender/features/sketch_tokens/top_im_patches/S1_prf0_feature0_ranked2.jpg
(21, 21)
[185, 39]
saving to /user_data/mmhender/features/sketch_tokens/top_im_patches/S1_prf0_feature0_ranked3.jpg
(21, 21)
[255, 96]
saving to /user_d